In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

print(os.getcwd()) 
print(os.listdir(os.getcwd()))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# %%writefile setting.py
# 样本图片目录
import os

SAMPLE_FILE_PATH = "../input/face-data/icml_face_data.csv"
print(SAMPLE_FILE_PATH)

# 分类数量
NUM_CLASSES = 7  

TRAIN_HDF5 = "./train.hdf5"
VAL_HDF5 = "./val.hdf5"
TEST_HDF5 = "./test.hdf5"

# 每批次样本数量
BATCH_SIZE = 128

# 项目输出文件保存目录
OUTPUT_PATH = "./"

# 数据集样本RGB平均值存位置及文件名称
DATASET_MEAN_FILE = OUTPUT_PATH + "/rgb_mean.json"

# 模型保存位置及文件名称
MODEL_FILE =  "./model.h5"


In [ ]:

# %%writefile EpochCheckpoint.py
import os
from tensorflow.keras.callbacks import Callback


# 模型存盘检查点，每训练5趟保存一次模型
class EpochCheckpoint(Callback):
    def __init__(self, output_path, every=5, start_at=0):
        # 调用父类的构造函数
        super(Callback, self).__init__()
        self.output_path = output_path  # 模型保存目录
        # 间隔趟数
        self.every = every
        # 起始趟数（当前趟数）
        self.start_epoch = start_at

    def on_epoch_end(self, epoch, logs={}):
        # 检查是否要向磁盘保存模型
        if (self.start_epoch + 1) % self.every == 0:
            p = os.path.sep.join([self.output_path,
                                  "epoch_{}.hdf5".format(self.start_epoch + 1)])
            self.model.save(p, overwrite=True)
        # 增加内部的趟数计数器
        self.start_epoch += 1


> ***EpochCheckpoint***

In [ ]:
# %%writefile ImageToArrayPreprocessor.py
# 图像样本维度重置预处理器
from tensorflow.keras.preprocessing.image import img_to_array

# 定义ImageToArrayPreprocessor类
class ImageToArrayPreprocessor:
    def __init__(self, data_format=None):
        # 保存图像数据的格式
        self.data_format = data_format

    def preprocess(self, image):
        """
        重置图像image的维度
        :param image: 要预处理的图像
        :return: 维度重置后的图像
        """

        # 调用tensorflow.keras的img_to_array方法正确重置图像的维度
        return img_to_array(image, data_format=self.data_format)


In [ ]:
# %%writefile HDF5DatasetWriter.py
# 定义HDF5DatasetWriter类
import os
import h5py

# HDF5数据集生成器
class HDF5DatasetWriter:

    # dims参数用来控制将要保存到数据集中的数据维度，类似于numpy数组的shape
    # 如果我们要保存扁平化的28×28 = 784 MNIST数据集原始像素数据，则dims = (70000，784)，
    # 因为NNIST数据集共有70000个样本，每个样本的维度是784。
    # 如果我们要存储原始CIFAR-10图像，则dims = (60000，32，32，3)
    # 因为CIFAR-10数据集共有60000图像，每个样本表示为一个 32×32×3 RGB图像。

    def __init__(self, dims, output_path, data_key="images", buf_size=1000):
        """

        :param dims: 用来控制将要保存到数据集中的数据维度，类似于numpy数组的shape。
        :param output_path: 生成的HDF5文件存储在磁盘上的路径
        :param data_key: 数据集的名称，默认值为"images"
        :param buf_size: 缓存大小，默认值1000

        """
        # 检查输出路径是否存在，如果存在则抛出异常
        if os.path.exists(output_path):
            raise ValueError("您提供的输出文件{}已经存在，请先手工删除！".format(output_path))
        # 创建并打开可写入HDF5文件
        # 然后在其中创建两个数据集
        # dataset : 类似数组组织的数据的集合，像 numpy 数组一样工作
        self.db = h5py.File(output_path, "w")  # 读取文件
        # 用于存储图像/特征
        self.data = self.db.create_dataset(data_key, dims, dtype="float")
        # 用于存储分类标签
        self.labels = self.db.create_dataset("labels", (dims[0],), dtype="int")

        # 保存缓存大小，然后初始化存缓和数据集索引
        self.buf_size = buf_size
        self.buffer = {"data": [], "labels": []}
        self.idx = 0

    def add(self, raw, label):
        """
        将数据和标签添加到缓存
        :param raw: 图像
        :param label: 对应的标签
        :return:
        """
        self.buffer["data"].extend(raw)
        self.buffer["labels"].extend(label)

        if len(self.buffer["data"]) >= self.buf_size:  # 缓存小桶盛满了，放入水池
            self.flush()  # 刷新缓冲区

    def flush(self):
        # 将缓存内容写入磁盘文件，然后清空缓存
        # 块状文件，顺序读写
        i = self.idx + len(self.buffer["data"])
        self.data[self.idx:i] = self.buffer["data"]
        self.labels[self.idx:i] = self.buffer["labels"]
        self.idx = i
        self.buffer = {"data": [], "labels": []}  # 清空缓存

    def store_class_labels(self, class_labels):
        # dataset 是类 numpy array 所以，你能写进的数据只能是数组
        # 创建一个数据集用来存储分类标签名称，然后保存分类标签
        dt = h5py.special_dtype(vlen=str)
        label_dim = (len(class_labels),)
        label_set = self.db.create_dataset("label_names", label_dim, dtype=dt)
        label_set[:] = class_labels

    def close(self):
        # 检查缓存中是否有记录，如果有，则必须写入磁盘文件
        if len(self.buffer["data"]) > 0:
            self.flush()

        # 关闭数据集
        self.db.close()


In [ ]:
# %%writefile HDF5DatasetGenerator.py
import h5py
import numpy as np
from keras.utils.np_utils import to_categorical

# 生成HDF5文件
class HDF5DatasetGenerator:
    def __init__(self, db_file, batch_size,
                 preprocessors=None, aug=None,
                 binarize=True, classes=2):
        """
        :param db_file: 数据集文件
        :param batch_size: 样本数量
        :param preprocessors: 预处理器列表
        :param aug: 数据增强处理器列表
        :param binarize: 标签是否二值化
        :param classes: 分类数，这里为二，我们只有猫狗两个分类
        """
        # 每批次样本数量
        self.batchSize = batch_size

        # 预处理器列表
        self.preprocessors = preprocessors

        # 数据增强处理器列表，可以使用 Keras ImageDataGenerator实现的数据增强算法
        self.aug = aug

        # 标签是否二值化，我们在HDF5数据集中保存的类别标签为单个整型的列表，
        # 然而，当我们使用分类交叉熵(categorical cross-entropy)或二值交叉熵(binary cross-entropy)
        # 作为计算损失的方法： 我们必须将标签二值化为独热编码向量组(one-hot encoded vectors)
        self.binarize = binarize

        # 不重复的类别数量，在计算标签二值化独热编码向量组时需要该值
        self.classes = classes

        # 打开HDF5数据集文件
        self.db = h5py.File(db_file,'r')

        # 数据集样本数量
        self.numImages = self.db["labels"].shape[0]

    def generator(self, passes=np.inf):
        # hdf5中的数据分批读取到内存中
        # 初始化训练趟数计数器
        epochs = 0

        # 执行无限循环，一旦达到规定的训练趟数,模型会自动停止训练
        while epochs < passes:
            # 遍历HDF5数据集
            for i in np.arange(0, self.numImages, self.batchSize):

                # 从HDF5数据集取出一批样本和标签
                images = self.db["images"][i:i + self.batchSize]
                labels = self.db["labels"][i:i + self.batchSize]

                # 检查标签是否有转化为独热编码向量组
                if self.binarize:
                    labels = to_categorical(labels, self.classes)

                # 如果有预处理器
                if self.preprocessors is not None:
                    # 初始化预处理结果图像列表
                    processed_images = []

                    # 遍历图像
                    for image in images:
                        # 遍历预处理器，对每个图像执行全部预处理
                        for p in self.preprocessors:
                            image = p.preprocess(image)

                        # 更新预处理结果图像列表
                        processed_images.append(image)

                    # 将图像数组更新为预处理结果图像
                    images = np.array(processed_images)

                # 如果指定了数据增强器，则实施之
                if self.aug is not None:
                    (images, labels) = next(self.aug.flow(images, labels,
                                                          batch_size=self.batchSize))

                # 生成图像和标记元组
                yield images, labels

            # 增加训练趟数计数器
            epochs += 1

    def close(self):
        # 关闭HDF5数据集
        self.db.close()


In [ ]:
# %%writefile TrainingMonitor.py
#TrainingMonitor类
from tensorflow.keras.callbacks import BaseLogger
import matplotlib.pyplot as plt
import numpy as np
import json
import os


# 定义TrainingMonitor类
class TrainingMonitor(BaseLogger):
    def __init__(self, fig_path, json_path=None, start_at=0):
        super(TrainingMonitor, self).__init__()
        self.fig_path = fig_path  # 绘图文件保存路径
        self.json_path = json_path  # Json文件保存路径
        self.start_at = start_at  # 开始的趟数
        self.history = {}  # 训练日志历史字典

    def on_train_begin(self, logs={}):

        # 若json日志文件存在，则加载训练日志字典
        if self.json_path is not None:
            if os.path.exists(self.json_path):
                self.history = json.loads(open(self.json_path).read())  # append 4 value after each epoch
                # 如果指定了训练趟数起点
                if self.start_at > 0:
                    # 遍历训练日志字典，解掉起点趟数之后的日志
                    for k in self.history.keys():
                        self.history[k] = self.history[k][:self.start_at]

    def on_epoch_end(self, epoch, logs={}):

        # 针对整个训练过程，遍历日志，更新训练损失、训练准确度等
        for (k, v) in logs.items():
            log = self.history.get(k, [])
            log.append(float(v))
            self.history[k] = log

        if self.json_path is not None:
            f = open(self.json_path, 'w')
            f.write(json.dumps(self.history, skipkeys=True))  # 序列化为json文件
            f.close()

        # 训练两趟后开始绘图
        if len(self.history["loss"]) > 1:
            # 绘图训练损失和准确度趋势图
            N = np.arange(0, len(self.history["loss"]))
            plt.style.use("ggplot")
            plt.figure()
            plt.plot(N, self.history["loss"], label="train_loss")
            plt.plot(N, self.history["val_loss"], label="val_loss")
            # plt.plot(N,self.history["acc"],label="train_acc")  # GPU version
            plt.plot(N, self.history["accuracy"], label="train_acc")  # CPU version
            # plt.plot(N,self.history["val_acc"],label="val_acc") # GPU version
            plt.plot(N, self.history["val_accuracy"], label="val_acc")  # CPU version
            epochs = len(self.history["loss"])
            plt.title("Training Loss & Accuracy [Epoch {}]".format(epochs))
            plt.xlabel("Epoch #")
            plt.ylabel("Loss/Accuracy")
            plt.legend()
            plt.savefig(self.fig_path)
            plt.close()


In [ ]:
# %%writefile build_hdf5.py
# 导入必要的包
import numpy as np
# import HDF5DatasetWriter
# import setting

print("[信息] 加载csv格式数据集文件...")

# 打开csv格式数据集文件
file = open(SAMPLE_FILE_PATH)

# 跳过第一行(表头）
file.__next__()

# 声明训练、校验、测试数据集
(train_images, train_labels) = ([], [])
(val_images, val_labels) = ([], [])
(test_images, test_labels) = ([], [])

# 存储数据集样本类别分布的字典
count_by_label_train = {}
count_by_label_val = {}
count_by_label_test = {}

# 遍历一遍文件的每一行
for row in file:
    # 提取每一行的标签(label),用途(usage)以及图像(image)。
    (label, usage, image) = row.strip().split(",")
    # 标签转整数
    label = int(label)
    # 将一维像素列表变维成48x48灰度图像
    image = np.array(image.split(" "), dtype="uint8")
    image = image.reshape((48, 48))

    # 如果用途为training，添加到训练样本集
    if usage == "Training":
        train_images.append(image)
        train_labels.append(label)
        count = count_by_label_train.get(label, 0)
        count_by_label_train[label] = count + 1
    # 如果用途为publicTest，添加到校验样本集
    elif usage == "PublicTest":
        val_images.append(image)
        val_labels.append(label)
        count = count_by_label_val.get(label, 0)
        count_by_label_val[label] = count + 1
        # 如果用途为PrivateTest，添加到测试样本集
    elif usage == "PrivateTest":
            test_images.append(image)
            test_labels.append(label)
            count = count_by_label_test.get(label, 0)
            count_by_label_test[label] = count + 1

# 关闭CVS样本文件
file.close()
# 输出训练、校验、测试样本数量
print("[信息] 训练样本数量：{}".format(len(train_images)))
print("[信息]校验样本数量：{}".format(len(val_images)))
print("[信息]测试样本数量：{}".format(len(test_images)))
# 输出训练、校验、测试样本数量分布
print(count_by_label_train)
print("[信息]校验样本分布：")
print(count_by_label_val)
print("[信息]测试样本分布：")
print(count_by_label_test)
# 构建一个训练、校验、测试数据集列表，
# 每个元素由数据集类型名称、全部样本文件名称、全部样本整型标签、HDF5输出文件构成
datasets = [(train_images, train_labels, TRAIN_HDF5),
(val_images, val_labels, VAL_HDF5),
(test_images, test_labels, TEST_HDF5)]

# 遍历数据集元组
for (images, labels, outputPath) in datasets:
    # 创建HDF5写入器
    print("[信息]构建{}...".format(outputPath))
    writer = HDF5DatasetWriter((len(images), 48, 48), outputPath)
    # 遍历每个图像，将其加入数据集
    for (image, label) in zip(images, labels):
        writer.add([image], [label])
    # 关闭HDF5写入器
    writer.close()


> ***生成hdf5文件***

In [ ]:
# %%writefile VGG11.py
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras import backend
from tensorflow.keras.regularizers import l1,l2


class VGG11:
    @staticmethod
    def build(width, height, channel, classes, reg=0.0002):
        """

        :param width: 输入样本的宽度
        :param height: 输入样本的高度
        :param channel: 输入样本的通道
        :param classes:分类数量
        :param reg:正则化因子
        :return:VGG网络模型
        """

        model = Sequential(name="VGG11")
        # 缺省输入格式为通道后首 ("channels-last")
        shape = (height, width, channel)

        channel_dimension = -1
        # 如果输入格式为通道前罱
        # 重新设首输入格式和通道位首指示
        if backend.image_data_format() == "channels_first":
            shape = (channel, height, width)
            channel_dimension = 1

        # 第一卷积块
        model.add(Conv2D(64, (3, 3), input_shape=shape, padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # 第二卷积块
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # 第三卷积块
        model.add(Conv2D(256, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Conv2D(256, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # 第四卷积块
        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # 第五卷积块
        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))
        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))
        model.add(MaxPooling2D(pool_size=(2, 2), padding="same", strides=(1, 1)))

        # 第一全连接层
        model.add(Flatten())
        model.add(Dense(256, kernel_regularizer=l2(reg)))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))

        # 第二全连接层
        model.add(Dense(128, kernel_regularizer=l2(reg)))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))

        # 第三全连接层
        model.add(Dense(classes, kernel_regularizer=l1(reg)))
        model.add(Activation("softmax"))

        return model


# 测 试 VGG11
if __name__ == '__main__':
    my_model = VGG11.build(width=48, height=48, channel=1, classes=7, reg=0.0002)
    print(my_model.summary())


> ***VGG_11***

In [ ]:
# %%writefile training.py
import argparse
import os

import matplotlib
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# from VGG11 import VGG11
# import setting
# from EpochCheckpoint import EpochCheckpoint
# from HDF5DatasetGenerator import HDF5DatasetGenerator
# from ImageToArrayPreprocessor import ImageToArrayPreprocessor
# from TrainingMonitor import TrainingMonitor


# 设置matplotlib可以在后台保存绘图
matplotlib.use("Agg")

# # # 构造命令行参数解析器
# ap = argparse.ArgumentParser()
# ap.add_argument("-c", "--checkpoints", required=True, help="检查点输出目录")
# ap.add_argument("-m", "--model", required=False, type=str, help="要加载的检查点模型文件")
# ap.add_argument("-s", "--start-epoch", type=int, default=0, help="重新训练的趟数起点")
# args = vars(ap.parse_args())

# 训练集数据增强生成器
train_aug = ImageDataGenerator(rotation_range=10,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               rescale=1 / 255.0,
                               fill_mode="nearest")

# 校验集数据增强生成器
val_aug = ImageDataGenerator(rescale=1 / 255.0)

# 初始化图像预处理器
iap = ImageToArrayPreprocessor()

# 初始化训练数据集生成器
train_gen = HDF5DatasetGenerator(TRAIN_HDF5,
                                 BATCH_SIZE,
                                 aug=train_aug,
                                 preprocessors=[iap],
                                 classes=NUM_CLASSES)

# 初始化校验数据集生成器
val_gen = HDF5DatasetGenerator(VAL_HDF5,
                               BATCH_SIZE,
                               aug=val_aug,
                               preprocessors=[iap],
                               classes=NUM_CLASSES)


print("[信息] 编译模型...")
# 初始化优化器
opt = Adam(lr=1e-3)
model = VGG11.build(width=48, height=48, channel=1, classes=NUM_CLASSES)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])


# 构造训练回调列表，这里主要是绘图回调
fig_path = os.path.sep.join([OUTPUT_PATH, "VGG11.png"])
# json_path = os.path.sep.join([OUTPUT_PATH, "VGG11.json"])
callbacks = [TrainingMonitor(fig_path=fig_path)]

# 训练网络
model.fit_generator(train_gen.generator(),
                    steps_per_epoch=train_gen.numImages // BATCH_SIZE,
                    validation_data=val_gen.generator(),
                    validation_steps=val_gen.numImages // BATCH_SIZE,
                    epochs=100,
                    max_queue_size=BATCH_SIZE * 2,
                    callbacks=callbacks,
                    verbose=1)
# 将训练得到的模型保存到文件
print("[信息] 保存模型...")
model.save(MODEL_FILE, overwrite=True)

# 关闭HDF5数据集
train_gen.close()
val_gen.close()



> ***train***

In [ ]:
# %%writefile evaluating.py
# import setting
# from ImageToArrayPreprocessor import  ImageToArrayPreprocessor
# from HDF5DatasetGenerator import HDF5DatasetGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

# 初始化图像预处理器
testAug = ImageDataGenerator(rescale=1 / 255.0)
iap = ImageToArrayPreprocessor()

# 初始化测试数据集生成器
testGen = HDF5DatasetGenerator(TEST_HDF5,
                               BATCH_SIZE,
                               aug = testAug,
                               preprocessors=[iap],
                               classes=NUM_CLASSES)

# 加载前面训练好的网络
print("[信息] 加载网络模型...")
model = load_model(MODEL_FILE)

# 评估网络模型
(loss, acc) = model.evaluate_generator(testGen.generator(),
                                       steps=testGen.numImages // BATCH_SIZE,
                                       max_queue_size=BATCH_SIZE * 2)
print("[信息] 测试集准确率: {:.2f}%".format(acc * 100))

# 关闭HDF5数据集
testGen.close()

In [ ]:
# %%writefile emotion_detector.py
import cv2
import imutils
import numpy as np

from tensorflow.keras.models import load_model

from tensorflow.keras.preprocessing.image import img_to_array

#加载opencv级联分类器

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#加训练好的表情识别网络模型

model = load_model('output/model.h5')
#面部表情分类

EMOTIONS = ['Angry','Disgust', 'Scared','Happy','Sad','Surprise','Neutral']
#打开第一个摄像头
capture = cv2.VideoCapture(0,cv2.CAP_DSHOW)

#持续采集摄像头图像帧
while True:
    ret,frame = capture.read()
    #帧图像缩小并转化为灰度图像
    frame = imutils.resize(frame,width=300)
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    #初始化显示使用的画布
    canvas = np.zeros((240,300,3),dtype="uint8")

    frameClone = frame.copy()

    rects = face_detector.detectMultiScale(
        gray,scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30,30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    if len(rects)>0:
        rect = sorted(rects,reverse=True,
                      key=lambda x:(x[2]-x[0])*(x[3]-x[1]))[0]

        #抠出脸部图像，预处理以适合分类网络模型的输入要求
        (fX,fY,fW,fH) = rect
        roi = gray[fY:fY+fH,fX:fX+fH] #灰度化
        roi =cv2.resize(roi,(48,48)) #缩小
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)    #转换为np数组
        roi = np.expand_dims(roi,axis=0)  #增加样本数量维度

        #使用模型进行推断
        predicts = model.predict(roi)[0]

        label = EMOTIONS[predicts.argmax()]

        cv2.putText(frameClone,label,(fX,fY-10),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
        cv2.rectangle(frameClone,(fX,fY),(fX+fW,fY+fH),
                      (0,0,255),1,cv2.LINE_AA)

        for (i,(emotion,prob)) in enumerate(zip(EMOTIONS,predicts)):
            text = "{}:{:.2f}%".format(emotion,prob*100)

            w = int(prob*300)
            cv2.rectangle(canvas,(5,(i*32)+5),
                          (5+w,(i*32)+32),(0,0,255),-1)
            cv2.putText(canvas,
                        text,(10,(i*32)+23),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (255,255,255),
                        1,
                        cv2.LINE_AA)

    cv2.imshow("Emotion Detection",frameClone)
    cv2.imshow("Result",canvas)

    if cv2.waitKey(1) == 27:
        break

capture.release()
cv2.destoryAllWindows()
